In [2]:
# import the libraries needed to represent the tsp and to embed it on the quantum annealer
from dwave.system.samplers import DWaveSampler
from dwave.system.composites import EmbeddingComposite
from neal import SimulatedAnnealingSampler

import numpy as np
import dimod
import itertools 
import minorminer
import networkx as nx
import dwave_networkx as dnx


In [25]:
# model parameters
n = 4
D = np.zeros((n,n))

D[0] = np.fromstring('0,1,1,1', dtype='int', sep=',')
D[1] = np.fromstring('1,0,1,1', dtype='int', sep=',')
D[2] = np.fromstring('1,1,0,1', dtype='int', sep=',')
D[3] = np.fromstring('1,1,1,0', dtype='int', sep=',')

print(f'D is: \n{D}')

# annealer parameters
num_reads = 100
TOKEN = ''

D is: 
[[0. 1. 1. 1.]
 [1. 0. 1. 1.]
 [1. 1. 0. 1.]
 [1. 1. 1. 0.]]


In [26]:
# normalization factors
A_NORMALIZATION = 1
B_NORMALIZATION = 1

In [27]:
from pyqubo import Binary, Array

x = {i:Binary("x_{}".format(i)) for i in range(n*n)}
x

{0: Binary(x_0),
 1: Binary(x_1),
 2: Binary(x_2),
 3: Binary(x_3),
 4: Binary(x_4),
 5: Binary(x_5),
 6: Binary(x_6),
 7: Binary(x_7),
 8: Binary(x_8),
 9: Binary(x_9),
 10: Binary(x_10),
 11: Binary(x_11),
 12: Binary(x_12),
 13: Binary(x_13),
 14: Binary(x_14),
 15: Binary(x_15)}

In [28]:
c_A = 0

H_a1 = 0
for v in range(n):
    c_A += 1
    H_temp = 1
    for j in range(v * n, v * n + n):
        H_temp -= x[j]
            
    H_a1 += H_temp * H_temp

H_a2 = 0
for j in range(n):
    c_A += 1
    H_temp = 1
    for v in range(n):
        H_temp -= x[j + v * n]

    H_a2 += H_temp * H_temp

c_A

8

In [29]:
H_b = 0

for u in range(n):
    for v in range(n):
        if u != v:
            for j in range(n):
                H_b += D[u,v] * x[u * n + j] * x[v * n + j]

In [30]:
H_A = A_NORMALIZATION * H_a1 + A_NORMALIZATION * H_a2
H_B = B_NORMALIZATION * H_b

H = H_A + H_B

model = H.compile()
qubo, _= model.to_qubo()

BQM = dimod.BinaryQuadraticModel.from_qubo(qubo)
BQM

BinaryQuadraticModel({x_13: -2.0, x_5: -2.0, x_3: -2.0, x_2: -2.0, x_15: -2.0, x_14: -2.0, x_1: -2.0, x_9: -2.0, x_8: -2.0, x_10: -2.0, x_0: -2.0, x_7: -2.0, x_12: -2.0, x_6: -2.0, x_11: -2.0, x_4: -2.0}, {('x_13', 'x_5'): 4.0, ('x_13', 'x_1'): 4.0, ('x_13', 'x_12'): 2.0, ('x_13', 'x_9'): 4.0, ('x_13', 'x_14'): 2.0, ('x_13', 'x_15'): 2.0, ('x_5', 'x_1'): 4.0, ('x_5', 'x_7'): 2.0, ('x_5', 'x_9'): 4.0, ('x_5', 'x_4'): 2.0, ('x_5', 'x_6'): 2.0, ('x_3', 'x_2'): 2.0, ('x_3', 'x_1'): 2.0, ('x_3', 'x_15'): 4.0, ('x_3', 'x_0'): 2.0, ('x_3', 'x_7'): 4.0, ('x_3', 'x_11'): 4.0, ('x_2', 'x_10'): 4.0, ('x_2', 'x_0'): 2.0, ('x_2', 'x_6'): 4.0, ('x_2', 'x_14'): 4.0, ('x_2', 'x_1'): 2.0, ('x_15', 'x_14'): 2.0, ('x_15', 'x_7'): 4.0, ('x_15', 'x_11'): 4.0, ('x_15', 'x_12'): 2.0, ('x_14', 'x_10'): 4.0, ('x_14', 'x_6'): 4.0, ('x_14', 'x_12'): 2.0, ('x_1', 'x_0'): 2.0, ('x_1', 'x_9'): 4.0, ('x_9', 'x_8'): 2.0, ('x_9', 'x_10'): 2.0, ('x_9', 'x_11'): 2.0, ('x_8', 'x_10'): 2.0, ('x_8', 'x_12'): 4.0, ('x_8', '

## Solve the problem with Simulated Annealing

In [31]:
sampler = SimulatedAnnealingSampler()

response_SA = sampler.sample(BQM, num_reads = num_reads)

In [32]:
response_SA

SampleSet(rec.array([([1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0], -8., 1),
           ([0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0], -8., 1),
           ([0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0], -8., 1),
           ([0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0], -8., 1),
           ([1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0], -8., 1),
           ([0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0], -8., 1),
           ([0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0], -8., 1),
           ([0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1], -8., 1),
           ([0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1], -8., 1),
           ([0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0], -8., 1),
           ([1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1], -8., 1),
           ([0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0], -8., 1),
           ([1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0], -8., 1),
           ([0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0], -8., 

In [33]:
def print_response_data(response):
    pos_sets = []
    # ------- Print results to user -------
    print('-' * 143)
    print('{:>45s}{:>60s}{:>21s}{:>12s}'.format('Set 0','Set 1','Energy',"Count"))
    print('-' * 143)
    for sample, E, occ in response.data(fields=['sample','energy',"num_occurrences"]):
        S0 = [k for k,v in sample.items() if v == 0]
        S1 = [k for k,v in sample.items() if v == 1]
        pos_sets.append(S1)
        print('{:>15s}{:>15s}{:^10s}{:^15s}'.format(str(S0),str(S1),str(E),str(occ)))
    
    return pos_sets

In [34]:
solution = print_response_data(response_SA.aggregate())

-----------------------------------------------------------------------------------------------------------------------------------------------
                                        Set 0                                                       Set 1               Energy       Count
-----------------------------------------------------------------------------------------------------------------------------------------------
['x_1', 'x_11', 'x_12', 'x_14', 'x_15', 'x_2', 'x_3', 'x_4', 'x_5', 'x_6', 'x_8', 'x_9']['x_0', 'x_10', 'x_13', 'x_7']   -8.0          4       
['x_1', 'x_10', 'x_11', 'x_12', 'x_13', 'x_15', 'x_2', 'x_3', 'x_4', 'x_5', 'x_6', 'x_8']['x_0', 'x_14', 'x_7', 'x_9']   -8.0          2       
['x_1', 'x_10', 'x_12', 'x_14', 'x_15', 'x_2', 'x_3', 'x_4', 'x_5', 'x_7', 'x_8', 'x_9']['x_0', 'x_11', 'x_13', 'x_6']   -8.0          3       
['x_0', 'x_1', 'x_10', 'x_11', 'x_13', 'x_14', 'x_15', 'x_3', 'x_4', 'x_5', 'x_6', 'x_8']['x_12', 'x_2', 'x_7', 'x_9']   -8.0          2     

In [35]:
def map_set(pos_set):
    mapping = []
    for var in pos_set:
        value = int(var.split('_')[1])
        mapping.append([int(value / n), value % n])
    
    return mapping

def return_solution(response, pos_solution):
    for p_set in pos_solution:
        res = map_set(p_set)
        s_res = sorted(res, key=lambda x: x[1])
        for i in range(n - 1):
            print((s_res[i])[0], end='-->')
        
        print((s_res[n - 1])[0])

print(f'There are {len(solution)} solutions')
return_solution(response_SA, solution)

There are 24 solutions
0-->3-->2-->1
0-->2-->3-->1
0-->3-->1-->2
3-->2-->0-->1
2-->1-->0-->3
1-->3-->0-->2
1-->0-->2-->3
0-->1-->2-->3
2-->3-->0-->1
2-->3-->1-->0
3-->0-->1-->2
2-->1-->3-->0
0-->2-->1-->3
1-->0-->3-->2
1-->2-->0-->3
1-->2-->3-->0
1-->3-->2-->0
3-->1-->0-->2
0-->1-->3-->2
2-->0-->1-->3
3-->0-->2-->1
3-->1-->2-->0
3-->2-->1-->0
2-->0-->3-->1
